In [6]:
import pandas as pd
import numpy as np
import pickle as pkl
from datetime import datetime
import utils
import math

### Load the dataset 

In [7]:
# Load the data from the database for Seattle that was scraped on 3/27
database = "../data/raw/Apartments.db"
conn = utils.create_connection(database)

df = pd.read_sql_query(
    """SELECT * FROM Apartments""",
    conn,
)
conn.close()

df.head()

,property_name,unit_id,rent,beds,baths,sqft,date_available,date_scraped,city,fitness_center,...,concierge,pool,garage,dogs_allowed,cats_allowed,income_restrictions,latitude,longitude,neighborhood,zipcode
0,Rooster Apartments,301,"$1,820",1,1,510,Now,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115
1,Rooster Apartments,604,"$1,835",1,1,510,Now,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115
2,Rooster Apartments,302,"$1,820",1,1,510,Apr. 3,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115
3,Rooster Apartments,520,"$1,825",1,1,567,Now,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115
4,Rooster Apartments,231,"$2,581",1,1,974,Now,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115


In [8]:
# Checking null and empty values
pd.concat([df.isnull().sum(),df.eq('').sum()],keys=['Nulls','Empty'],axis=1)

,Nulls,Empty
property_name,0,0
unit_id,0,0
rent,0,0
beds,0,0
baths,0,0
sqft,0,132
date_available,0,0
date_scraped,0,0
city,0,0
fitness_center,0,0


### Cleaning the Beds and Baths columns

In [9]:
df.beds.value_counts()

1         2061
1 bed     1493
2          902
0          825
2 beds     677
Studio     596
3 beds     128
3           76
4           60
4 beds      53
5 beds      22
5           22
6 beds       7
8 beds       4
7 beds       2
Name: beds, dtype: int64

In [10]:
df.baths.value_counts()

1 bath       2224
1            2061
2             902
0             825
2 baths       563
3              76
1.5 baths      66
4              60
3 baths        48
2.5 baths      34
5              22
3.5 baths      14
5 baths        13
4 baths        12
0.5 bath        8
Name: baths, dtype: int64

In [11]:
# Convert studio to zero bedrooms and remove non-numeric characters
df.beds = pd.to_numeric(df.beds.str.replace('Studio','0').str.replace('\D','', regex = True)).astype(int)

# Remove non-numeric characters from the baths column
df.baths = df.baths.str.replace('[a-zA-z]','', regex = True).astype(float)

In [12]:
df.beds.value_counts()

1    3554
2    1579
0    1421
3     204
4     113
5      44
6       7
8       4
7       2
Name: beds, dtype: int64

In [13]:
df.baths.value_counts()

1.0    4285
2.0    1465
0.0     825
3.0     124
4.0      72
1.5      66
5.0      35
2.5      34
3.5      14
0.5       8
Name: baths, dtype: int64

### Cleaning the Rent and SQFT column

In [14]:
# Dropping rows where rent isn't listed or is blank
df.drop(df.loc[df.rent.str.contains('Call')].index, axis = 0, inplace = True)
df.drop(df.loc[df.rent == ''].index, axis = 0, inplace = True)

# Remove comma, dollar sign and /Person from Rent
df.rent = df.rent.str.replace('$','', regex = True).str.replace(',','', regex = True).str.replace('/ Person','').str.replace('/Person','').str.replace(r'\r+|\n+|\t+','', regex=True).str.replace(' ','')

# Remove comma and sq ft from SQFT column
df.sqft = df.sqft.str.replace('sq ft','', regex = True).str.replace(',','', regex = True)

In [15]:
pd.concat([df.isnull().sum(),df.eq('').sum()],keys=['Nulls','Empty'],axis=1)

,Nulls,Empty
property_name,0,0
unit_id,0,0
rent,0,0
beds,0,0
baths,0,0
sqft,0,55
date_available,0,0
date_scraped,0,0
city,0,0
fitness_center,0,0


In [16]:
# Average out rent or SQFT values where a range is listed
# For example: $1,200 - $1,300 becomes 1,250
def get_average(strings):
    try:
        lowValue = int(strings[0])
        highValue = int(strings[-1])
        return int((lowValue + highValue)/2)
    except:
        return strings

df.rent = np.where(df.rent.str.contains("–"),
                         (df.rent.str.split("–")).map(get_average),
                         df.rent)

df.sqft = np.where(df.sqft.str.contains("–"),
                         (df.sqft.str.split("–")).map(get_average),
                         df.sqft)


In [17]:
df.head()

,property_name,unit_id,rent,beds,baths,sqft,date_available,date_scraped,city,fitness_center,...,concierge,pool,garage,dogs_allowed,cats_allowed,income_restrictions,latitude,longitude,neighborhood,zipcode
0,Rooster Apartments,301,1820,1,1.0,510,Now,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115
1,Rooster Apartments,604,1835,1,1.0,510,Now,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115
2,Rooster Apartments,302,1820,1,1.0,510,Apr. 3,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115
3,Rooster Apartments,520,1825,1,1.0,567,Now,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115
4,Rooster Apartments,231,2581,1,1.0,974,Now,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115


#### Impute the missing values in the sqft column

In [18]:
# Impute the missing values in the sqft column by using a simple regression model of rent vs. sqft
from sklearn.linear_model import LinearRegression

# Repalce empty values with nan 
df.sqft.replace('', np.nan, inplace=True)

In [19]:
pd.concat([df.isnull().sum(),df.eq('').sum()],keys=['Nulls','Empty'],axis=1)

,Nulls,Empty
property_name,0,0
unit_id,0,0
rent,0,0
beds,0,0
baths,0,0
sqft,55,0
date_available,0,0
date_scraped,0,0
city,0,0
fitness_center,0,0


In [20]:
df[df.sqft.isnull()].head()

,property_name,unit_id,rent,beds,baths,sqft,date_available,date_scraped,city,fitness_center,...,concierge,pool,garage,dogs_allowed,cats_allowed,income_restrictions,latitude,longitude,neighborhood,zipcode
29,"1 br, 1 bath House - 2021 NE 96th St Priva...",2021 NE 96th St,850,1,1.0,NaN,Apr. 1,2022-03-28,Seattle,0,...,0,0,0,0,0,0,47.69826,-122.30586,Maple Leaf,WA98115
202,6011 Roosevelt Way NE,6011 Roosevelt Way NE,3200,0,1.0,NaN,Available Now,2022-03-28,Seattle,0,...,0,0,0,0,0,0,47.6727,-122.31753,Roosevelt,WA98115
331,450 South Main St Unit 1109,1109,4700,2,2.0,NaN,Available Now,2022-03-28,Seattle,0,...,0,0,0,0,0,0,47.60025,-122.32806,Chinatown,WA98104
378,1 bedroom in Seattle WA 98125,1,1275,1,1.0,NaN,Available Now,2022-03-28,Seattle,0,...,0,0,0,0,0,0,47.73081,-122.29418,Lake City,WA98125
551,2 bedroom in Seattle WA 98125,22,1795,2,2.0,NaN,Available Now,2022-03-28,Seattle,0,...,0,0,0,0,0,0,47.73081,-122.29418,Lake City,WA98125


In [21]:
# Create a seperate dataframe without the nan values
df_model = df.dropna()

X = np.array(df_model.rent.astype(float)).reshape(-1,1)
y = np.array(df_model.sqft.astype(float)).reshape(-1,1)

model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [22]:
# Make predictions on NaN sqft columns
def make_prediction(row):
    row = np.array(float(row)).reshape(-1,1)
    return int(model.predict(row))

df.sqft = df.apply(lambda row: make_prediction(row.rent) if pd.isnull(row.sqft) else row.sqft, axis =1)

In [23]:
df.iloc[809,:].head(6)

property_name    The Martin
unit_id                1702
rent                   2045
beds                      0
baths                   0.0
sqft                    537
Name: 898, dtype: object

### Clean and adjust the date_available column
* Date will be converted to date time format

In [24]:
# Dropping units that are Not Available
df.drop(df.loc[df.date_available.str.contains('Not Available')].index, axis = 0, inplace = True)

# Set all units listed as Available or Soon to zero days until available
df.date_available = df.date_available.str.replace(r'Available Now','Now').str.replace(r'Available Soon','Now').str.replace('Soon','Now')

# Remove periods from abbreviated months
df.date_available = df.date_available.str.replace('.','',regex=True)

In [25]:
def adjust_date_available(dateString):
    if dateString == 'Now':
        return datetime.today().strftime('%Y-%m-%d')
    current_year = datetime.now().year
    clean_date = dateString.split(',')[0]+f', {current_year}'
    date = datetime.strptime(clean_date, '%b %d, %Y')
    date = date.strftime('%Y-%m-%d')
    return date

df.date_available = df.date_available.map(adjust_date_available)
df.head()

,property_name,unit_id,rent,beds,baths,sqft,date_available,date_scraped,city,fitness_center,...,concierge,pool,garage,dogs_allowed,cats_allowed,income_restrictions,latitude,longitude,neighborhood,zipcode
0,Rooster Apartments,301,1820,1,1.0,510,2022-03-28,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115
1,Rooster Apartments,604,1835,1,1.0,510,2022-03-28,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115
2,Rooster Apartments,302,1820,1,1.0,510,2022-04-03,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115
3,Rooster Apartments,520,1825,1,1.0,567,2022-03-28,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115
4,Rooster Apartments,231,2581,1,1.0,974,2022-03-28,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,WA98115


### Clean the zipcode colulmn

In [26]:
# Remove non-numeric characters from the zipcode column
df.zipcode = df.zipcode.str.replace('[a-zA-z]','', regex = True).astype(str)

In [27]:
df.head()

,property_name,unit_id,rent,beds,baths,sqft,date_available,date_scraped,city,fitness_center,...,concierge,pool,garage,dogs_allowed,cats_allowed,income_restrictions,latitude,longitude,neighborhood,zipcode
0,Rooster Apartments,301,1820,1,1.0,510,2022-03-28,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,98115
1,Rooster Apartments,604,1835,1,1.0,510,2022-03-28,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,98115
2,Rooster Apartments,302,1820,1,1.0,510,2022-04-03,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,98115
3,Rooster Apartments,520,1825,1,1.0,567,2022-03-28,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,98115
4,Rooster Apartments,231,2581,1,1.0,974,2022-03-28,2022-03-28,Seattle,1,...,0,0,1,1,1,0,47.67617,-122.3181,Seattle,98115


### Add additional features
* Distance to downtown Seattle
* Distance to transit stop

In [28]:
# Calculate the distance to downtown seattle

downtown_seattle = {"lat": 47.604013, "lon": -122.335167}

df['dist_seattle']=df.apply(lambda row: utils.calculate_distance(float(row.latitude), float(row.longitude), downtown_seattle['lat'], downtown_seattle['lon']), axis=1)
df.head()

,property_name,unit_id,rent,beds,baths,sqft,date_available,date_scraped,city,fitness_center,...,pool,garage,dogs_allowed,cats_allowed,income_restrictions,latitude,longitude,neighborhood,zipcode,dist_seattle
0,Rooster Apartments,301,1820,1,1.0,510,2022-03-28,2022-03-28,Seattle,1,...,0,1,1,1,0,47.67617,-122.3181,Seattle,98115,5.048482
1,Rooster Apartments,604,1835,1,1.0,510,2022-03-28,2022-03-28,Seattle,1,...,0,1,1,1,0,47.67617,-122.3181,Seattle,98115,5.048482
2,Rooster Apartments,302,1820,1,1.0,510,2022-04-03,2022-03-28,Seattle,1,...,0,1,1,1,0,47.67617,-122.3181,Seattle,98115,5.048482
3,Rooster Apartments,520,1825,1,1.0,567,2022-03-28,2022-03-28,Seattle,1,...,0,1,1,1,0,47.67617,-122.3181,Seattle,98115,5.048482
4,Rooster Apartments,231,2581,1,1.0,974,2022-03-28,2022-03-28,Seattle,1,...,0,1,1,1,0,47.67617,-122.3181,Seattle,98115,5.048482


In [29]:
# Open the train stop location data
train_stops = pd.read_csv("../data/external/seattle_train_stops.txt")[
    ["stop_id", "stop_lat", "stop_lon"]
]

# Open the bus stop location data
bus_stops = pd.read_csv("../data/external/seattle_bus_stops.txt")[
    ["stop_id", "stop_lat", "stop_lon"]
]

In [30]:
# Calculate the distances to the bus and train stops for each apartment complex
df['distBus']=df.apply(lambda row: utils.calculate_distance(float(row.latitude), float(row.longitude), bus_stops['stop_lat'], bus_stops['stop_lon']).min(), axis=1)
df['distTrain']=df.apply(lambda row: utils.calculate_distance(float(row.latitude), float(row.longitude), train_stops['stop_lat'], train_stops['stop_lon']).min(), axis=1)

In [31]:
df['dist_transit'] = np.where(df.distBus < df.distTrain,
                              df.distBus,
                              df.distTrain)

df.drop(columns={'distBus', 'distTrain'}, inplace=True)
df.head()

,property_name,unit_id,rent,beds,baths,sqft,date_available,date_scraped,city,fitness_center,...,garage,dogs_allowed,cats_allowed,income_restrictions,latitude,longitude,neighborhood,zipcode,dist_seattle,dist_transit
0,Rooster Apartments,301,1820,1,1.0,510,2022-03-28,2022-03-28,Seattle,1,...,1,1,1,0,47.67617,-122.3181,Seattle,98115,5.048482,0.029716
1,Rooster Apartments,604,1835,1,1.0,510,2022-03-28,2022-03-28,Seattle,1,...,1,1,1,0,47.67617,-122.3181,Seattle,98115,5.048482,0.029716
2,Rooster Apartments,302,1820,1,1.0,510,2022-04-03,2022-03-28,Seattle,1,...,1,1,1,0,47.67617,-122.3181,Seattle,98115,5.048482,0.029716
3,Rooster Apartments,520,1825,1,1.0,567,2022-03-28,2022-03-28,Seattle,1,...,1,1,1,0,47.67617,-122.3181,Seattle,98115,5.048482,0.029716
4,Rooster Apartments,231,2581,1,1.0,974,2022-03-28,2022-03-28,Seattle,1,...,1,1,1,0,47.67617,-122.3181,Seattle,98115,5.048482,0.029716


### Cleaning all of the feature columns

In [34]:
# Check that a unit doesn't have a 1 for both in unit washer/dryer and laundry facilities
df[(df['in_unit_washer_dryer'] == 1) & (df['laundry_facilities'] == 1)][['property_name', 'in_unit_washer_dryer', 'laundry_facilities']]

,property_name,in_unit_washer_dryer,laundry_facilities
9,Puget Vista,1,1
10,Puget Vista,1,1
11,Puget Vista,1,1
12,Puget Vista,1,1
40,7429 Rainier Ave S,1,1
...,...,...,...
6920,Cascade,1,1
6921,Cascade,1,1
6922,Cascade,1,1
6923,Cascade,1,1


In [35]:
# Change any listing with both in unit washer/dryer and laundry facilities to only in unit washer/dryer
df.laundry_facilities = np.where((df['in_unit_washer_dryer'] == 1) & (df['laundry_facilities'] == 1),
                                 0,
                                 df.laundry_facilities)

In [36]:
# Combined the dog and cat columns into a single Pet column

df['pets_allowed']=np.where((df.dogs_allowed == 1) | (df.cats_allowed == 1),
                            1,
                            0)

# Drop the dog and cat columns
df.drop(columns={'dogs_allowed', 'cats_allowed'}, inplace=True)

In [37]:
# Check if the unit ID has MFTE in the title and set the income_restrictions to 1 if MFTE is found
df.income_restrictions = np.where(df.unit_id.str.contains('MFTE'), 1, df.income_restrictions)

In [38]:
df.head()

,property_name,unit_id,rent,beds,baths,sqft,date_available,date_scraped,city,fitness_center,...,pool,garage,income_restrictions,latitude,longitude,neighborhood,zipcode,dist_seattle,dist_transit,pets_allowed
0,Rooster Apartments,301,1820,1,1.0,510,2022-03-28,2022-03-28,Seattle,1,...,0,1,0,47.67617,-122.3181,Seattle,98115,5.048482,0.029716,1
1,Rooster Apartments,604,1835,1,1.0,510,2022-03-28,2022-03-28,Seattle,1,...,0,1,0,47.67617,-122.3181,Seattle,98115,5.048482,0.029716,1
2,Rooster Apartments,302,1820,1,1.0,510,2022-04-03,2022-03-28,Seattle,1,...,0,1,0,47.67617,-122.3181,Seattle,98115,5.048482,0.029716,1
3,Rooster Apartments,520,1825,1,1.0,567,2022-03-28,2022-03-28,Seattle,1,...,0,1,0,47.67617,-122.3181,Seattle,98115,5.048482,0.029716,1
4,Rooster Apartments,231,2581,1,1.0,974,2022-03-28,2022-03-28,Seattle,1,...,0,1,0,47.67617,-122.3181,Seattle,98115,5.048482,0.029716,1


### Save the dataset to CSV

In [39]:
df.to_csv('../data/interim/seattle_03_28_2022.csv', index = False)